# Операции над объектами Pandas

На прошлом занятии мы начали изучать библиотеку Pandas и познакомились с ее основными структурами данных: `Index`, `Series` и `DataFrame`. Очевидно, что работа с данными не ограничивается исключительно их хранением и чтением, а зачастую включается в себя применение специальных операций и функций: логических, арифметических, математических, статистических и т.д. в зависимости от задачи. Сегодня мы с вами познакомимся с функциями и методами объектов Pandas, которые позволяют эффективно обрабатывать большие объемы различных данных.

**Необходимые импорты**:

In [518]:
import timeit

import numpy as np
import pandas as pd
import seaborn as sns

## Векторизованные операции в стиле NumPy

Pandas во многом построена на массивах NumPy, а потому значительная часть векторизованных операций из NumPy может быть использована вместе с объектами `pd.Series` и `pd.DataFrame`. Однако, при использовании этих операций с объектами `Pandas` существуют свои тонкости. Рассмотрим эти тонкости детальнее.

### Сохранение индексов

Объекты `pd.Series` и `pd.DataFrame` - это не просто одномерные и двумерные массивы данных. Это структуры данных с явными типизированными индексами. Этот факт учитывается во время выполнения различных операций над объектами `pd.Series` и `pd.DataFrame`.  Так при выполнении унарных операций или при применении к этим объектам арифметических функций, Pandas будет использовать индексы исходного объекта в качестве индексов результата выполнения вычислений. Благодаря этому подходу разработчики получают легкий способ обновлять столбцы датафреймов, используя результаты выполнения различных операций.

Рассмотрим примеры сохранения индексов во время выполнения различных вычислений. 

In [519]:
series = pd.Series(
    data=np.random.normal(size=5),
    index=list("ABCDE"),
)
series_exp = np.exp(series)

print(
    f"Original series:\n{series}",
    f"Series exp:\n{series_exp}",
    sep="\n\n",
)

Original series:
A    0.002208
B    1.078762
C   -0.493503
D    0.572021
E    0.047031
dtype: float64

Series exp:
A    1.002211
B    2.941036
C    0.610484
D    1.771844
E    1.048155
dtype: float64


Аналогичным образом сохранение индексов строк и индексов столбцов выполняется и для `pd.DataFrame`.

In [520]:
row_amount, col_amount = 3, 4

data_frame = pd.DataFrame(
    data=np.random.normal(size=(row_amount, col_amount)),
    index=[f"row_{i + 1}" for i in range(row_amount)],
    columns=[f"col_{i + 1}" for i in range(col_amount)],
)
data_frame

,col_1,col_2,col_3,col_4
row_1,0.076237,-0.606260,0.360982,-1.912847
row_2,2.033542,0.194428,-0.118304,0.424014
row_3,-0.008096,-0.391975,-0.377495,-0.298360


In [521]:
np.sin(data_frame * np.pi)

,col_1,col_2,col_3,col_4
row_1,0.237223,-0.944796,0.906136,0.270391
row_2,0.105181,0.573533,-0.363166,0.971642
row_3,-0.025432,-0.942965,-0.926851,-0.805977


### Выравнивание индексов

Вторая, более важная особенность выполнения операций с объектами Pandas, заключается в выравнивании индексов. Из NumPy мы знаем, что при попытке выполнениях бинарных операций с одномерными массивами `np.ndarray`, мы получим ошибку. Поскольку массивы NumPy лежат в основе библиотеки Pandas, может создастся впечатление, что похожее поведение должно быть справедливо и для объектов `pd.Series` с разными индексами. Ведь `pd.Series` - это аналог одномерного массива `np.ndarray` с явно заданным индексом, а отличия в количестве элементов массивов `np.ndarray` аналогично отличиям в индексах объектов `pd.Series`.  Однако это ложное предположение. В Pandas возможно выполнение бинарных операций над объектами `pd.Series` с разными индексами. Более того, в результате этих операций получаются вполне предсказуемые и логичные результаты. Рассмотрим пример.

In [522]:
populations = pd.Series(
    {
        "Moscow": 13149803,
        "Saint Petersburg": 5600044,
        "Novosibirsk": 1635338,
        "Ekaterinburg": 1539371,
    },
)
areas = pd.Series(
    {
        "Kazan": 515.8,
        "Ekaterinburg": 1112,
        "Moscow": 2511,
    },
)
population_density = populations / areas

print(f"Population density:\n{population_density}")

Population density:
Ekaterinburg        1384.326439
Kazan                       NaN
Moscow              5236.878933
Novosibirsk                 NaN
Saint Petersburg            NaN
dtype: float64


В данном примере мы определили `pd.Series` `populations`, в котором отражены численности населения российских городов. Также мы определили `pd.Series` `areas`, в котором отражены площади российских городов. Индексы определенных серий разные. Они имеют пересечения в виде ключей `"Ekaterinburg"` и `"Moscow"`, однако остальные ключи отличаются. Далее выполняется операция бинарного деления, чтобы определить плотность населения в российских городах. В результате выполнения мы получили объект `pd.Series`, в котором плотность населения определена только для тех значений индекса, которые присутствовали и в первой, и во второй серии. Остальным же значениям индекса соответствует странное значение `NaN`, о котором мы поговорим ниже. Однако, факт остается фактом - мы можем выполнять бинарные операции с объектами, обладающими разными индексами.

Рассмотрим детальнее результат выполнения такой операции. Во-первых, видно что индекс результирующей серии является объединением значений индексов операндов:

In [523]:
is_index_united = np.all(
    population_density.index == populations.index.union(areas.index)
)

print(f"is index was united: {is_index_united}")

is index was united: True


Во-вторых, значения индекса результата выполнения операции отсортированы в порядке возрастания. Поскольку в данном примере в качестве значений индекса были использованы названия городов, значения результирующего индекса отсортированы в алфавитном порядке:

In [524]:
population_density.index

Index(['Ekaterinburg', 'Kazan', 'Moscow', 'Novosibirsk', 'Saint Petersburg'], dtype='object')

Аналогичные результаты будут справедливы и для объектов `pd.DataFrame`. Единственное отличие будет заключать в выравнивании индексов по двум измерениям.

In [525]:
data_frame1 = pd.DataFrame(
    data=np.random.randint(0, 20, size=(2, 2)),
    columns=list("AB"),
)
data_frame1

,A,B
0,7,5
1,9,7


In [526]:
data_frame2 = pd.DataFrame(
    data=np.random.randint(0, 10, size=(3, 3)),
    columns=list("BAC"),
)
data_frame2

,B,A,C
0,3,5,8
1,9,9,7
2,4,5,2


In [527]:
data_frame1 + data_frame2

,A,B,C
0,12.0,8.0,NaN
1,18.0,16.0,NaN
2,NaN,NaN,NaN


Единственный момент, который может смущать нас на данном этапе - наличие непонятного значения `NaN` в данных. При выполнении бинарных операций с объектами Pandas, мы можем предотвратить появление этого значения в результате. Чтобы это сделать, нам придется воспользоваться другой формой бинарных операций - бинарными операциями в форме методов объектов `pd.Series` и `pd.DataFrame`. При использовании данной формы бинарных операций с помощью аргумента `fill_value` мы можем явно указать, какое значение стоит использовать в тех случаях, когда ключ отсутствует в одном из операндов. В этом случае результат будет выглядеть так:

In [528]:
data_frame1.add(data_frame2, fill_value=0)

,A,B,C
0,12.0,8.0,8.0
1,18.0,16.0,7.0
2,5.0,4.0,2.0


Таблица соответствия методов и бинарных операций:

| Оператор Python | Метод объекта Pandas |
|---|---|
| + | add() |
| - | sub(), subtract() |
| * | mul(), multiply() |
| / | truediv(), div(), divide() |
| // | floordiv() |
| % | mod() |
| ** | pow() |

### Транслирование (Broadcasting)

Во всех предыдущих примерах для выполнения бинарных операций мы использовали операнды одних и тех же типов данных. Однако мы можем выполнять бинарные операции с операндами различных типов данных. Например, мы можем вычитать объект типа `pd.Series` из объекта типа `pd.DataFrame`. В случае, если индексы объектов совпадают, результат будет аналогичен вычитанию одномерного массива `np.ndarray` из двумерного массива `np.ndarray`. Т.е. будет происходить транслирование одномерного объекта по уже знакомым нам правилам:

In [529]:
row_amount, col_amount = 3, 4

data_frame = pd.DataFrame(
    data=np.random.randint(0, 10, size=(row_amount, col_amount)),
    index=[f"row_{i + 1}" for i in range(row_amount)],
    columns=[f"col_{i + 1}" for i in range(col_amount)],
)
data_frame

,col_1,col_2,col_3,col_4
row_1,2,3,1,2
row_2,7,2,8,5
row_3,5,0,9,7


In [530]:
data_frame - data_frame.loc["row_2"]

,col_1,col_2,col_3,col_4
row_1,-5,1,-7,-3
row_2,0,0,0,0
row_3,-2,-2,1,2


По умолчанию вычитание происходит построчно. Однако далеко не всегда мы хотим выполнять операции построчно. Существуют случае, когда нам необходимо выполнить некоторую операцию по столбцам. В этом случае нам придется воспользоваться методами объектов Pandas для выполнения бинарных операций, а также указать значение аргумента `axis=0`.

In [531]:
data_frame.subtract(data_frame["col_1"], axis=0)

,col_1,col_2,col_3,col_4
row_1,0,1,-1,0
row_2,0,-5,1,-2
row_3,0,-5,4,2


Также далеко не всегда индексы операндов могут совпадать. В этом случае будет происходить уже знакомое нам выравнивание.

In [532]:
first_row_odd_columns = data_frame.iloc[0, ::2]

print(
    "even columns values from first row:\n"
    f"{first_row_odd_columns}"
)

even columns values from first row:
col_1    2
col_3    1
Name: row_1, dtype: int32


In [533]:
data_frame - first_row_odd_columns

,col_1,col_2,col_3,col_4
row_1,0.0,NaN,0.0,NaN
row_2,5.0,NaN,7.0,NaN
row_3,3.0,NaN,8.0,NaN


## NaN. Обработка отсутствующих данных

### Что такое NaN и откуда он взялся?

В предыдущих примерах мы столкнулись со значением `NaN` в наших данных. Это специальное значение, которое Pandas использует для того, чтобы помечать отсутствующие данные. `NaN` - акроним, составленный из первых букв фразы *not a number*. Давайте разберемся, как еще `NaN` может попасть в наши данные, как задать это значение самостоятельно, и как с ним работать.

Часто при работе с реальными данными, нам приходится сталкиваться с неполными данными. Например, в результате некоторого статистического опроса несколько респондентов забыли заполнить графу возраста. В этом случае некоторые данные о респондентах будут неполные. Неполные данные легко представить на бумаге, но как представлять неполные данные в коде?  В Python для этих целей мы использовали объект-синглтон None. Однако при работе с NumPy использование None приводит к печальным последствиям. Рассмотрим пример.

In [534]:
array = np.array([1, 2, None, 4])

print(
    f"array data:\n{array}",
    f"array dtype: {array.dtype}",
    sep="\n\n",
)

array data:
[1 2 None 4]

array dtype: object


Из этого кода следует следующее. Если NumPy встречает в данных объект `None`, он осуществляет повышающее преобразование типов. После преобразования типов в массиве `np.ndarray` будут лежать данные типа `object`. Т.е. NumPy будет воспринимать содержимое  массива, как обычные объекты Python. Это значит, что работа с таким массивом не будет отличаться от работы с обычными списками Python. Никакая векторизация в таком случае невозможна.

In [535]:
setup = "import numpy as np"
template = "np.arange(int(1e6), dtype=%s).sum()"
iteration_amount = 1000

for dtype in ["np.object_", "np.int32"]:
    print(f"dtype: {dtype}")
    time_per_iter = timeit.timeit(
        setup=setup,
        stmt=template % dtype,
        number=iteration_amount,
    ) / iteration_amount
    print(f"time_taken: {time_per_iter:.4f}s;", end="\n\n")

dtype: np.object_
time_taken: 0.0715s;

dtype: np.int32
time_taken: 0.0020s;



Более того, мы даже не сможем выполнять некоторые агрегирующие операции над содержимым такого массива. Из-за наличия в данных `None` мы с большой долей вероятности получим ошибку.

In [536]:
# print(f"array:\n{array}")
# sum_of_elements = array.sum()

Чтобы обойти все эти ограничения, в NumPy реализовано специальное сигнальное значение `np.nan` в соответствии со стандартом IEEE. Фактически `np.nan` - это специальное число с плавающей точкой, используемое в качестве признака отсутствия данных. 

In [537]:
array = np.array([1, 2, np.nan, 3])

print(
    f"array data:\n{array}",
    f"array dtype: {array.dtype}",
    sep="\n\n",
)

array data:
[ 1.  2. nan  3.]

array dtype: float64


По своей природе значение `np.nan` похоже на вирус, который заражает собой любые данные, т.к. при выполнении любой операции с использованием `np.nan` результат также будет представлять собой `np.nan`.

In [538]:
print(
    f"addition: {np.nan + 42}",
    f"multiplication: {np.nan * 42}",
    f"agregation: {array.sum()}",
    f"safe agregation: {np.nansum(array)}",
    sep="\n",
)

addition: nan
multiplication: nan
agregation: nan
safe agregation: 6.0


Поскольку в основе Pandas лежит NumPy, мы можем использовать значение `np.nan`, чтобы помечать отсутствующие данные. Также разработчики Pandas добавили возможность использования `None` для того, чтобы помечать отсутствующие данные. Таким образом мы имеем два взаимозаменяемых способа задания пропущенных значений.

In [539]:
series = pd.Series([1, None, 2, np.nan])
print(f"series:\n{series}")

series:
0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64


Также обращаем ваше внимание, что при появлении в данных `NaN`, Pandas может произвести повышающее преобразование типов. Так, в данном примеры изначальный тип данных объекта `pd.Series` был `int32`, т.е. тип данных был целочисленный. После того, как мы пометили данные под индексом 0 как отсутствующие, Pandas произвел повышающее преобразование типов до `float64`. Это произошло, потому что `np.nan` - это число с плавающей точкой, а объект `None`, используемый для пометки отсутствующих данных, Pandas неявно преобразует в `np.nan`.

In [540]:
series = pd.Series(np.arange(4))
print(f"original series:\n{series}", end="\n\n")

series[0] = None
print(f"corrupted series:\n{series}")

original series:
0    0
1    1
2    2
3    3
dtype: int64

corrupted series:
0    NaN
1    1.0
2    2.0
3    3.0
dtype: float64


### Выявление пустых значений

Первое, что стоит сделать с полученными данными - определить, присутствуют ли в них пропуски или нет. И если пропуски присутствуют, желательно понимать, где именно. Определить положения `NaN` в данных можно с помощью метода `isnull`.

In [541]:
series = pd.Series([1, None, 2, 3, np.nan])
print(f"series:\n{series}")

series:
0    1.0
1    NaN
2    2.0
3    3.0
4    NaN
dtype: float64


In [542]:
mask_data_missed = series.isnull()
print(f"missed data mask:\n{mask_data_missed}")

missed data mask:
0    False
1     True
2    False
3    False
4     True
dtype: bool


В Pandas также определен антипод метода `isnull` - `notnull`, который позволяет определить булеву маски для данных, не являющихся `NaN`.

In [543]:
print(
    f"corrupted data:\n{series[mask_data_missed]}",
    f"correct data:\n{series[series.notnull()]}",
    sep="\n\n",
)

corrupted data:
1   NaN
4   NaN
dtype: float64

correct data:
0    1.0
2    2.0
3    3.0
dtype: float64


### Удаление пустых значений

Установив наличие `NaN` в данных, необходимо решить, что с ними делать. Редко в каких задачах уместно оставлять пропущенные данные. Обычно от пропусков или избавляются, или пытаются их заполнить по определенным правилам. Если данных очень много, пропусков очень мало, а их заполнение нецелесообразно, может потребоваться простое удаление таких данных из Pandas объектов. Это можно сделать с помощью метода `dropna`. 

In [544]:
series = pd.Series([1, None, 2, 3, np.nan])
print(f"series:\n{series}")

series:
0    1.0
1    NaN
2    2.0
3    3.0
4    NaN
dtype: float64


In [545]:
print(
    f"correct data:\n{series.dropna()}",
)

correct data:
0    1.0
2    2.0
3    3.0
dtype: float64


В случае работы с объектом `pd.Series` удаление `NaN` довольно прямолинейно. Однако при удалении данных из `pd.DataFrame` возникают сложности. Дело в том, что мы не можем удалять из датафрейма отдельные ячейки с данными. Можно удалить только строку или столбец целиком. По умолчанию `dropna` удаляет строки, содержащие хотя бы одно значение `NaN`.

In [546]:
data_frame = pd.DataFrame(
    data=[
        [1, 2, np.nan],
        [4, 5, 6],
        [np.nan, 8, np.nan],
    ],
    columns=list("ABC")
)
data_frame

,A,B,C
0,1.0,2,NaN
1,4.0,5,6.0
2,NaN,8,NaN


In [547]:
data_frame.dropna()

,A,B,C
1,4.0,5,6.0


Используя аргумент `axis`, мы можем указать измерение, вдоль которого должно анализироваться наличие пропусков. В примере ниже будут удалены все столбцы, содержащие хотя бы одно значение `NaN`.

In [548]:
data_frame.dropna(axis="columns")

,B
0,2
1,5
2,8


Так же функция `dropna` позволяет настраивать стратегии удаления строк и столбцов из датафрейма. Как говорилось выше, по умолчанию для удаления строки или столбца достаточно наличия хотя бы одного значения `NaN`. Однако такая стратегия может быть не всегда уместной. Часто в нашем распоряжении не так много данных, чтобы мы могли позволить себе выбрасывать строки или столбцы только из-за наличия одного `NaN`. Именно поэтому Pandas позволяет настроить правила, в соответствии с которым будет происходить удаление строк или столбцов. Так мы можем установить минимальное число значений отличных от `NaN`, необходимое для сохранения строки или столбца в датафрейме. Сделать это можно с помощью параметра `thresh`.

In [549]:
data_frame.dropna(thresh=1)

,A,B,C
0,1.0,2,NaN
1,4.0,5,6.0
2,NaN,8,NaN


Также при необходимости мы можем удалять только те строки или столбцы, которые полностью заполнены `NaN`.

In [550]:
data_frame.iloc[-1, 1] = np.nan
data_frame

,A,B,C
0,1.0,2.0,NaN
1,4.0,5.0,6.0
2,NaN,NaN,NaN


In [551]:
data_frame.dropna(how="all")

,A,B,C
0,1.0,2.0,NaN
1,4.0,5.0,6.0


### Заполнение пустых значений

Выше упоминалось, что в нашем распоряжении обычно не так много данных, чтобы мы могли позволить себе их сокращение путем удаления. Именно поэтому часто более оптимальной стратегией для обработки пропусков является заполнение, а не удаление. Заполнить пропущенные данные в Pandas можно с помощью методов `fillna`, `ffill` и `bfill`.

С помощью метода `fillna` можно заполнить недостающие данные переданным значением.

In [552]:
series = pd.Series([None, 1, 2, 3, np.nan])
print(
    f"series original:\n{series}",
    f"series filled:\n{series.fillna(0)}",
    sep="\n\n",
)

series original:
0    NaN
1    1.0
2    2.0
3    3.0
4    NaN
dtype: float64

series filled:
0    0.0
1    1.0
2    2.0
3    3.0
4    0.0
dtype: float64


`ffill` и `bfill` используют, соответственно, предшествующее и следующее значение для заполнения пропусков. 

In [553]:
print(
    f"series original:\n{series}",
    f"series forward fill:\n{series.ffill()}",
    f"series backward fill:\n{series.bfill()}",
    sep="\n\n",
)

series original:
0    NaN
1    1.0
2    2.0
3    3.0
4    NaN
dtype: float64

series forward fill:
0    NaN
1    1.0
2    2.0
3    3.0
4    3.0
dtype: float64

series backward fill:
0    1.0
1    1.0
2    2.0
3    3.0
4    NaN
dtype: float64


В случае с `DataFrame` справедливо все, сказанное выше.

In [554]:
data_frame = pd.DataFrame(
    [
        [1, 2, np.nan],
        [4, 5, 6],
        [np.nan, 9, np.nan],
    ],
    columns=list("ABC"),
)
data_frame

,A,B,C
0,1.0,2,NaN
1,4.0,5,6.0
2,NaN,9,NaN


In [555]:
data_frame.fillna(0)

,A,B,C
0,1.0,2,0.0
1,4.0,5,6.0
2,0.0,9,0.0


Мы также можем использовать объект `pd.Series`, чтобы заполнять пропущенные значения в разных столбцах по-разному.

In [556]:
fill_values = pd.Series(
    data=[42, 69],
    index=list("AC"),
)
data_frame.fillna(fill_values)

,A,B,C
0,1.0,2,69.0
1,4.0,5,6.0
2,42.0,9,69.0


Из-за двумерной природы объекта `pd.DataFrame` при использовании методов `ffill` и `bfill` мы можем выбирать размерность, вдоль которой будут использоваться предшествующие и следующие значения для заполнения пропусков. По умолчанию пропуски заполняются вдоль столбцов.

In [557]:
data_frame.ffill()

,A,B,C
0,1.0,2,NaN
1,4.0,5,6.0
2,4.0,9,6.0


In [558]:
data_frame.ffill(axis="columns")

,A,B,C
0,1.0,2.0,2.0
1,4.0,5.0,6.0
2,NaN,9.0,9.0


## Операции агрегирования

Объекты Pandas также поддерживают операции агрегирования. Поскольку простые операции агрегирования ничем не отличаются от соответствующих операций в NumPy, не будет заострять на них наше внимание.

In [559]:
planets_data = sns.load_dataset("planets")
planets_data.sample(n=5)

,method,number,orbital_period,mass,distance,year
445,Radial Velocity,1,2597.000000,1.880,33.01,2004
186,Transit,1,1.327347,NaN,317.00,2011
1004,Transit,1,1.749780,NaN,140.00,2012
368,Radial Velocity,1,3.971000,0.197,59.70,2002
187,Transit,1,2.797436,NaN,411.00,2011


In [560]:
print(
    f"mean orbital period: {planets_data['orbital_period'].mean():.2f};",
    f"first year of research: {planets_data['year'].min()};",
    f"last year of research: {planets_data['year'].max()};",
    sep="\n",
)

mean orbital period: 2002.92;
first year of research: 1989;
last year of research: 2014;


С помощью метода `unique` возможно получение уникальных значений, хранящихся в объекте типа `pd.Series`. 

In [561]:
planets_data["method"].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

Также в Pandas реализован метод `describe`, который позволяет вычислить основные статистики числовых данных. Эта функция может быть полезна при проведении разведывательного анализа данных.

In [562]:
planets_data.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


## Операции со строковыми данными

Обсуждая NumPy, мы с вами занимались исключительно работой с числовыми данными. Теперь, работая с таблицами Pandas, нам нередко будут встречаться строковые данные. Необходимо уметь эффективно обрабатывать строковые данные и уметь пользоваться векторизованными строковыми операциями. Это можно сделать, используя специальный атрибут объектов `pd.Series` и `pd.DataFrame` - `str`.

In [563]:
names = pd.Series(
    data=["john", "Paul", "george", "RINGO"],
)

print(
    f"names:\n{names}",
    f"names corrected:\n{names.str.capitalize()}",
    sep="\n\n",
)

names:
0      john
1      Paul
2    george
3     RINGO
dtype: object

names corrected:
0      John
1      Paul
2    George
3     Ringo
dtype: object


В Pandas реализовано множество векторизованных операций для работы со строками, являющееся аналогом множества методов строк в Python. Если вы знакомы с оригинальными методами, вы без труда сможете разобраться с логикой работы аналогов в Pandas. Ниже приведем пару примеров.

In [564]:
names = pd.Series(
    data=[
        "John Lennon",
        "Paul MacCartney",
        "George Harrison",
        "Ringo Starr",
    ],
)

print(
    f"names lens:\n{names.str.len()}",
    f"correction mask:\n{names.str.istitle()}",
    f"names split:\n{names.str.split()}",
    sep="\n\n",
)

names lens:
0    11
1    15
2    15
3    11
dtype: int64

correction mask:
0     True
1    False
2     True
3     True
dtype: bool

names split:
0        [John, Lennon]
1    [Paul, MacCartney]
2    [George, Harrison]
3        [Ringo, Starr]
dtype: object


Также в Pandas реализованы векторизованные операции для работы с регулярными выражениями.

In [565]:
print(
    f"pattern matching:\n{names.str.match(r'[A-Za-z]+')}",
    f"pattern inclusion:\n{names.str.contains(r'[Jj]ohn')}",
    f"pattern finding:\n{names.str.findall(r'^[^AEIOUY]*[^aeiouy]$')}",
    sep="\n\n",
)

pattern matching:
0    True
1    True
2    True
3    True
dtype: bool

pattern inclusion:
0     True
1    False
2    False
3    False
dtype: bool

pattern finding:
0        [John Lennon]
1                   []
2    [George Harrison]
3        [Ringo Starr]
dtype: object


Также реализованы операции векторизованного среза строк.

In [566]:
print(
    f"explicit slice:\n{names.str.slice(0, 3)}",
    f"implicit slice:\n{names.str[:3]}",
    sep="\n\n",
)

explicit slice:
0    Joh
1    Pau
2    Geo
3    Rin
dtype: object

implicit slice:
0    Joh
1    Pau
2    Geo
3    Rin
dtype: object


## Задача 1. My heart will go on

Датасет **titanic** из библиотеки `Seaborn` содержит информацию о пассажирах легендарного корабля Титаник, который затонул в 1912 году после столкновения с айсбергом. Этот набор данных часто используется для обучения и тестирования алгоритмов машинного обучения, особенно в задачах бинарной классификации (выжил / не выжил).

**Описание данных**

| Поле         | Тип      | Описание |
|--------------|----------|----------|
| `survived`   | int      | Выжил (1) или не выжил (0) |
| `pclass`     | int      | Класс билета (1, 2, 3) |
| `sex`        | str      | Пол (`male`/`female`) |
| `age`        | float    | Возраст |
| `sibsp`      | int      | Количество братьев/сестёр/супругов на борту |
| `parch`      | int      | Количество родителей/детей на борту |
| `fare`       | float    | Стоимость билета |
| `embarked`   | str      | Порт посадки (`C`=Cherbourg, `Q`=Queenstown, `S`=Southampton) |
| `class`      | str      | Класс билета (`First`, `Second`, `Third`) |
| `who`        | str      | Категория: `man`, `woman` или `child` |
| `adult_male` | bool     | Является ли взрослым мужчиной |
| `deck`       | str      | Палуба |
| `embark_town`| str      | Название порта посадки |
| `alive`      | str      | Выжил (`yes`/`no`) |
| `alone`      | bool     | Путешествовал один |

**Загрузка датасета**

In [578]:
titanic_data = sns.load_dataset("titanic")
titanic_data.sample(5)
titanic_data.shape

(891, 15)

**Задача**

Ниже описаны 10 небольших заданий, которые вам необходимо решить.

**Подсказка**:

В некоторых заданиях вам может быть полезен метод `value_counts`.

### Часть 1

Определите число пропущенных данных для каждого столбца таблицы `titanic_data`.

In [579]:
def count_spaces(data: pd.DataFrame):
    return data.isnull().sum()

count_spaces(titanic_data)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

### Часть 2

Удалите все столбцы, количество пропусков в которых превышает половину количества строк в таблице.

После того, как вы удалите все столбцы, нарушающие описанное условие, удалите все строки, количество элементов в которых превышает половину количества столбцов.

In [580]:
def del_nans(data: pd.DataFrame):
    data = data.dropna(thresh=data.shape[0]/2, axis='columns')
    data = data.dropna(thresh=data.shape[1]/2)
    return data

titanic_data = del_nans(titanic_data)
titanic_data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


### Часть 3

Если вы сделали все правильно, больше всего пропусков должно остаться в столбце `"age"` - 177. Их необходимо заполнить. Заполним пропуски следующим образом:
- Если значение столбца `"who"="man"`, пропуск необходимо заполнить медианным значением известных возрастов мужчин, округленным до ближайшего целого числа;
- Если значение столбца `"who"="woman"`, пропуск необходимо заполнить медианным значением известных возрастов женщин, округленным до ближайшего целого числа;
- Если значение столбца `"who"="child"`, пропуск необходимо заполнить медианным значением известных возрастов детей, округленным до ближайшего целого числа;

In [581]:
def fill_by_mean(data: pd.DataFrame): 
    mean_man = np.mean(data['age'][data['who']=='man'])
    mean_woman = np.mean(data['age'][data['who']=='woman'])
    mean_child = np.mean(data['age'][data['who']=='child'])

    data.loc[(data['who']=='man') & (data['age'].isnull()), ['age']] = mean_man
    data.loc[(data['who']=='woman') & (data['age'].isnull()), ['age']] = mean_woman
    data.loc[(data['who']=='child') & (data['age'].isnull()), ['age']] = mean_child
    return data

titanic_data = fill_by_mean(titanic_data)
titanic_data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,32.0,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


### Часть 4

Удалите все строки, в которых осталось больше одного пропуска. Если вы все сделали правильно, после этого действия в таблице не должно остаться пропусков.

In [582]:
def del_str_with_nans(data: pd.DataFrame):
    data = data.dropna(thresh=data.shape[1]-1)
    return data

titanic_data = del_str_with_nans(titanic_data)
titanic_data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,32.0,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


### Часть 5

Определите название города, из которого отправилось больше всего пассажиров.

In [583]:
def find_city_by_passengers_amount(data: pd.DataFrame):
    S_count = [(data['embark_town'] == 'Southampton').sum(), 'Southampton']
    C_count = [(data['embark_town'] == 'Cherbourg').sum(), 'Cherbourg']
    Q_count = [(data['embark_town'] == 'Queenstown').sum(), 'Queenstown']
    return max([S_count, C_count, Q_count])[1]

print(find_city_by_passengers_amount(titanic_data))

Southampton


### Часть 6

Определите процент выживших пассажиров от числа пассажиров, оставшихся в таблице после очистки данных. Ответ округлите до 2 знаков после запятой.

In [584]:
def find_alive_percent(data: pd.DataFrame):
    surv_peop = (data['survived'] == 1).sum()
    return np.round(surv_peop*100/data.shape[0], decimals=2)

print(find_alive_percent(titanic_data))

38.25


### Часть 7

Определите число выживших пассажиров для каждого пункта отправления. В ответе должен получиться объект типа `pd.Series`, индексы которого - названия пунктов отправления, а значения - число выживших пассажиров.

In [585]:
def find_pas_amount_per_city(data: pd.DataFrame):
    S_surv_count = ((data['embarked'] == 'S') & (data['survived'])).sum()
    C_surv_count = ((data['embarked'] == 'C') & (data['survived'])).sum()
    Q_surv_count = ((data['embarked'] == 'Q') & (data['survived'])).sum()

    surv_per_town = pd.Series(
        {
            "Southampton": S_surv_count,
            "Cherbourg": C_surv_count,
            "Queenstown": Q_surv_count
        }
    )
    return surv_per_town

find_pas_amount_per_city(titanic_data)

Southampton    217
Cherbourg       93
Queenstown      30
dtype: int64

### Часть 8

Определите процент выживших пассажиров в каждом классе. Значения округлите до 2 знаков после запятой. В ответе должен получиться объект типа `pd.Series`, индексы которого - названия классов, а значения - процент выживших пассажиров.

In [586]:
def find_alive_percent_by_class(data: pd.DataFrame):
    First_surv_count = ((data['class'] == 'First') & (
        data['survived'])).sum()
    Second_surv_count = ((data['class'] == 'Second') & (
        data['survived'])).sum()
    Third_surv_count = ((data['class'] == 'Third') & (
        data['survived'])).sum()

    First_count = (data['class'] == 'First').sum()
    Second_count = (data['class'] == 'Second').sum()
    Third_count = (data['class'] == 'Third').sum()

    First_surv_percent = np.round(First_surv_count*100/First_count, decimals=2)
    Second_surv_percent = np.round(Second_surv_count*100/Second_count, decimals=2)
    Third_surv_percent = np.round(Third_surv_count*100/Third_count, decimals=2)


    surv_per_class = pd.Series(
        {
            "First": First_surv_percent,
            "Second": Second_surv_percent,
            "Third": Third_surv_percent
        }
    )
    return surv_per_class

find_alive_percent_by_class(titanic_data)

First     62.62
Second    47.28
Third     24.24
dtype: float64

### Часть 9

Будем считать, что пассажиры, купившие билет **НЕ МЕНЕЕ** чем за $100, считаются богатыми. Определите процент выживших среди богатых пассажиров. Ответ округлите до 2 знаков после запятой. В ответе должно получиться число. 

In [587]:
def find_rich_alive_percent(data: pd.DataFrame):
    Rich_peop_count = ((data['fare'] >= 100.0)).sum()
    Rich_surv_count = ((data['fare'] >= 100.0) & (data['survived'])).sum()
    return np.round(Rich_surv_count*100/Rich_peop_count, decimals=2)

print(find_rich_alive_percent(titanic_data))

73.58


### Часть 10

Определите количество детей, путешествовавших в одиночку.

In [588]:
def find_alove_children(data: pd.DataFrame):    
    Alone_child_count = ((data['who'] == 'child') & (data['alone'])).sum()
    return Alone_child_count

print(find_alove_children(titanic_data))

6


Какие выводы вы можете сделать о выживших пассажирах Титаника? 